# Reading Avro Blobs Into Parquet Data Sets

### Dependency Importing and Environment Variable Retrieval

In [ ]:
import os
import string
import json
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession, SQLContext, Window
from pyspark.sql.functions import udf, mean, lit, stddev, col, expr, when, date_sub, avg, window
from pyspark.sql.types import DoubleType, ArrayType, ShortType, LongType, IntegerType, TimestampType, StructType, StringType, StructField
from pyspark.storagelevel import StorageLevel
from azureml.api.schema.dataTypes import DataTypes
from azureml.api.schema.sampleDefinition import SampleDefinition
from azureml.api.realtime.services import generate_schema
from azure.storage.table import TableService, Entity, TablePermissions

STORAGE_ACCOUNT_SUFFIX = 'core.windows.net'
TELEMETRY_STORAGE_ACCOUNT_NAME = os.getenv('TELEMETRY_STORAGE_ACCOUNT_NAME')
TELEMETRY_STORAGE_ACCOUNT_KEY = os.getenv('TELEMETRY_STORAGE_ACCOUNT_KEY')
TABLE_STORAGE_ACCOUNT_NAME = os.getenv('TABLE_STORAGE_ACCOUNT_NAME') #TODO need to add to ENV Variables
TABLE_STORAGE_ACCOUNT_KEY = os.getenv('TABLE_STORAGE_ACCOUNT_KEY') #TODO need to add to ENV Variables
TELEMETRY_CONTAINER_NAME = os.getenv('TELEMETRY_CONTAINER_NAME')
#LOGS_ARCHIVE_CONTAINER_NAME = 'logs-archive' #TODO: introduce environment variables
STAGING_STORAGE_ACCOUNT_NAME = os.getenv('STAGING_STORAGE_ACCOUNT_NAME')
STAGING_STORAGE_ACCOUNT_KEY = os.getenv('STAGING_STORAGE_ACCOUNT_KEY')
LOG_TABLE_NAME = os.getenv('LOG_TABLE_NAME') #TODO need to add this to ENV Variables

#### Development Variables

In [ ]:
#For development purposes only until ENV Variables get set
#STORAGE_ACCOUNT_SUFFIX = 'core.windows.net'
#TELEMETRY_STORAGE_ACCOUNT_NAME = "stguwsjc4vojbrbw"
#TELEMETRY_STORAGE_ACCOUNT_KEY = "2GGDWVBtGBy+hAgl5a1uGT4NeU2zzFdocuDFKnOwR2vc5wEOP7jTfbS3/Nl5vvzEudJ4nfH6ozmSOSPXo3xETA=="
#TABLE_STORAGE_ACCOUNT_NAME = "stgayjvm36mrwkec"
#TABLE_STORAGE_ACCOUNT_KEY = "RqR0In7OctP0BZhvFFNUctcIvxPU5XU14k+qBlYbY4lZzaBB1vVvKRwrjbWhdKtA+NylpK/8yeDH12Yb1w/PwA=="
#TELEMETRY_CONTAINER_NAME = "telemetry"
#LOG_TABLE_NAME = 'logs'
#LOGS_ARCHIVE_CONTAINER_NAME = "logs" #TODO: introduce environment variables
#STAGING_STORAGE_ACCOUNT_NAME = "stguwsjc4vojbrbw"
#STAGING_STORAGE_ACCOUNT_KEY = "2GGDWVBtGBy+hAgl5a1uGT4NeU2zzFdocuDFKnOwR2vc5wEOP7jTfbS3/Nl5vvzEudJ4nfH6ozmSOSPXo3xETA=="


### Setting up Drop Folder

In [ ]:
from pathlib import Path
data_dir = str(Path.home()) + '/data'

#TODO: Convert data_dir into env variable
% rm -rf $data_dir
% mkdir $data_dir $data_dir/logs

### Retrieving telemetry data (as spark dataframe)

In [ ]:
wasbTelemetryUrl = "wasb://{0}@{1}.blob.{2}/*/*/*/*/*/*/*".format(TELEMETRY_CONTAINER_NAME, 
                                                                  TELEMETRY_STORAGE_ACCOUNT_NAME, 
                                                                  STORAGE_ACCOUNT_SUFFIX)

sc = SparkSession.builder.getOrCreate()
hc = sc._jsc.hadoopConfiguration()
hc.set("avro.mapred.ignore.inputs.without.extension", "false")
if TELEMETRY_STORAGE_ACCOUNT_KEY:
     hc.set("fs.azure.account.key.{}.blob.core.windows.net".format(TELEMETRY_STORAGE_ACCOUNT_NAME), TELEMETRY_STORAGE_ACCOUNT_KEY)
hc.set("fs.azure.account.key.{}.blob.core.windows.net"
    .format(STAGING_STORAGE_ACCOUNT_NAME), STAGING_STORAGE_ACCOUNT_KEY)
sql = SQLContext.getOrCreate(sc)
avroblob = sql.read.format("com.databricks.spark.avro").load(wasbTelemetryUrl)
avroblob.show()

### Convert byteformatted "body" of raw blob data into JSON, then explode result into new Pyspark DataFrame

In [ ]:
#Convert byteformat to string format in pyspark dataframe
from json import loads as Loads
column = avroblob['Body']
string_udf = udf(lambda x: x.decode("utf-8"))
avroblob=avroblob.withColumn("BodyString", string_udf(column))
avroblob.printSchema()

#Convert "body" into new DataFrame
telemetry_df = sql.read.json(avroblob.select("BodyString").rdd.map(lambda r: r.BodyString))
telemetry_df.show()

In [ ]:
#columns to retain: timestamp, ambient_pressure, ambient_temperature machineID, pressure, speed, 
#                   speed_desired, temperature
subsetted_df = telemetry_df.select(["timestamp", "ambient_pressure","ambient_temperature","machineID","pressure","speed","speed_desired","temperature"])


In [ ]:
#modify timestamp format
import datetime
e = '%Y-%m-%dT%H:%M:%S.%f'
timestamp_udf = udf(lambda date: datetime.datetime.strptime(date, e), TimestampType())
reformatted_time_df = subsetted_df.withColumn("timestamp", timestamp_udf(subsetted_df["timestamp"]))

reformatted_time_df.printSchema()

### Write dataframe to Parquet in system storage

In [ ]:
reformatted_time_df.write.parquet(data_dir+"/telemetry", mode="overwrite")

## Get Logs

In [ ]:
#table retrieval
table_service = TableService(account_name=TABLE_STORAGE_ACCOUNT_NAME, account_key=TABLE_STORAGE_ACCOUNT_KEY)
tblob = table_service.query_entities(LOG_TABLE_NAME)

### Process log table data into Pandas DataFrame

In [ ]:
attributes = list()
for row in tblob:
    if (len(attributes) == 0):
        for attribute in row:
            attributes.append(attribute)
    break
log_df = pd.DataFrame(columns=attributes)
for row in tblob:
    row_dict = {}    
    for attribute in row:
        if (attribute != "Timestamp"):
            row_dict[attribute] = row[attribute]
        else:
            newtime = row[attribute].replace(tzinfo=None)
            timeitem = pd.Timestamp(newtime, tz=None)
            row_dict[attribute] = timeitem
    log_df = log_df.append(row_dict, ignore_index=True)
log_df.head()

### Select necessary attributes

In [ ]:
log_df = log_df[["Timestamp", "Code", "Level", "PartitionKey"]]
log_df.columns = ["timestamp", "code","level","machineID"]
log_df.index = log_df['timestamp']
del log_df['timestamp']
log_df.head()

### Write logs to system storage

In [ ]:
log_df.to_parquet(data_dir+"/logs/logs.parquet", engine='fastparquet', times='int96')